## Setup and Import Library

In [8]:
! pip install wandb
! pip install pytorch-lightning torchmetrics

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 248.8 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 190.7 kB/s eta 0:00:00a 0:00:01
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0


In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
import pytorch_lightning as pl
from torchvision import transforms
from torchmetrics.functional import accuracy
import torchvision.models as models
from torchvision.datasets import ImageFolder
import os
from torch.utils.data import random_split, DataLoader
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import wandb
from pytorch_lightning.loggers import WandbLogger
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import datetime

## Load Dataset



In [2]:
# Dataset

def get_dataloader(data_dir, batch_size=32, val_split=0.2):
  transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  ])

  dataset = ImageFolder(root = data_dir, transform=transform)

  val_size = int(len(dataset)* val_split)
  train_size = len(dataset) - val_size

  train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader


## Model

In [3]:
# Model

class ImageClassifier(pl.LightningModule):
  def __init__(self, num_classes):
    super().__init__()
    self.model = models.resnet18(pretrained=True)
    self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    self.criterion = nn.CrossEntropyLoss()
    self.num_classes = num_classes

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_idx):
    images, labels = batch
    outputs = self(images)
    loss = self.criterion(outputs, labels)
    acc = accuracy(outputs, labels, task='multiclass', num_classes=self.num_classes)
    self.log('train_loss', loss)
    self.log('train_acc', acc)
    return loss
  

  def validation_step(self, batch, batch_idx):
    images, labels = batch

    
    outputs = self(images)
    loss = self.criterion(outputs, labels)
    acc = accuracy(outputs, labels, task='multiclass', num_classes=self.num_classes)
    self.log('val_loss', loss)
    self.log('val_acc', acc)

    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
    return optimizer

## Training

In [47]:
# Training

wandb.login()
data_dir = 'dataset-original/'
num_classes = 6
batch_size = 16


train_loader, val_loader = get_dataloader(data_dir=data_dir, batch_size=batch_size)

model = ImageClassifier(num_classes=num_classes)

checkpoint_callback = ModelCheckpoint(
    monitor= 'val_loss',
    dirpath = 'checkpoint/',
    filename ='Trash-Classification-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

wandb_logger = WandbLogger(
    project='Trash-Classification',
    name = 'training-run',
    save_dir = 'logs/'
)

wandb_logger.experiment.config.update({
    "data_dir": data_dir,
    "num_classes":num_classes,
    "batch_size":batch_size,
    "max_epochs":20
})

trainer = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback, early_stopping],
    logger=wandb_logger
)
# Resume training dari checkpoint
trainer.fit(
    model, 
    train_loader, 
    val_loader,
    ckpt_path='checkpoint/Trash-Classification-epoch=16-val_loss=0.26.ckpt'  # Ganti dengan path checkpoint terakhir
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at checkpoint/Trash-Classification-epoch=16-val_loss=0.26.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNet           | 11.2 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.718    Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode
Restored all states from the checkpoint at checkpoint/Trash-Classification-epoch=16-val_loss=0.26.ckpt


Epoch 19: 100%|██████████| 127/127 [03:34<00:00,  0.59it/s, v_num=n5c8]    

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 127/127 [03:34<00:00,  0.59it/s, v_num=n5c8]


## Evaluate

In [4]:
data_dir = 'dataset-original/'
num_classes = 6
batch_size = 16

_, val_loader = get_dataloader(data_dir=data_dir, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ImageClassifier.load_from_checkpoint('checkpoint/Trash-Classification.ckpt', num_classes=num_classes) #Replace with your model
model.to(device)
model.eval()

acc = 0.0
with torch.no_grad():
    for batch in val_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        acc += (outputs.argmax(dim=1) == labels).float().mean().item()
print(f"Validation Accuracy: {acc/len(val_loader):.4f}")

/home/najwatsabita/miniconda3/envs/najwavenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/najwatsabita/miniconda3/envs/najwavenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Validation Accuracy: 0.9727


## Inference

In [5]:
# Data cleaning from .DS_Store
def clean_dataset_folders(data_dir):
    """Remove .DS_Store and other hidden files from dataset folders"""
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.startswith('.'):  
                os.remove(os.path.join(root, file))

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_valid_classes(data_dir):
    """Get valid class names excluding hidden files"""
    return sorted([d for d in os.listdir(data_dir) 
                  if os.path.isdir(os.path.join(data_dir, d)) and not d.startswith('.')])

def setup_classifier(data_dir, checkpoint_path, num_classes=6):
    clean_dataset_folders(data_dir)
    
    class_names = get_valid_classes(data_dir)
    if len(class_names) != num_classes:
        raise ValueError(f"Expected {num_classes} classes but found {len(class_names)}: {class_names}")
    
    idx_to_class = {idx: class_name for idx, class_name in enumerate(class_names)}
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = ImageClassifier.load_from_checkpoint(checkpoint_path, num_classes=num_classes)
    model.to(device)
    model.eval()
    
    return model, idx_to_class, device

def predict_image(image_path, model, idx_to_class, device):
    """Predict class for a single image"""
 
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        probabilities = torch.softmax(output, dim=1)
        predicted_idx = output.argmax(dim=1).item()
        confidence = probabilities[0][predicted_idx].item()
        
    predicted_class = idx_to_class[predicted_idx]
    
    return {
        'class': predicted_class,
        'confidence': confidence,
        'index': predicted_idx
    }

if __name__ == "__main__":
    data_dir = 'dataset-original/'
    model_path = 'checkpoint/Trash-Classification.ckpt'
    
    model, idx_to_class, device = setup_classifier(data_dir, model_path)
    
    print("Available classes:", list(idx_to_class.values()))
    
    image_path = 'glass.jpg'
    result = predict_image(image_path, model, idx_to_class, device)
    
    print(f"Predicted Class: {result['class']}")
    print(f"Confidence: {result['confidence']:.2%}")

Available classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Predicted Class: glass
Confidence: 92.41%


In [6]:

def clean_dataset_folders(data_dir):
    """Remove .DS_Store and other hidden files from dataset folders"""
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.startswith('.'):  
                os.remove(os.path.join(root, file))

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_valid_classes(data_dir):
    """Get valid class names excluding hidden files"""
    return sorted([d for d in os.listdir(data_dir) 
                  if os.path.isdir(os.path.join(data_dir, d)) and not d.startswith('.')])

def setup_classifier(data_dir, checkpoint_path, num_classes=6):
    clean_dataset_folders(data_dir)
    
    class_names = get_valid_classes(data_dir)
    if len(class_names) != num_classes:
        raise ValueError(f"Expected {num_classes} classes but found {len(class_names)}: {class_names}")
    
    idx_to_class = {idx: class_name for idx, class_name in enumerate(class_names)}
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = ImageClassifier.load_from_checkpoint(checkpoint_path, num_classes=num_classes)
    model.to(device)
    model.eval()
    
    return model, idx_to_class, device

def draw_prediction_box(image, prediction, confidence):
    """Draw bounding box and prediction text on the image"""
    draw = ImageDraw.Draw(image)
    
    width, height = image.size
    
    box_margin = 0.1 
    box_left = int(width * box_margin)
    box_top = int(height * box_margin)
    box_right = int(width * (1 - box_margin))
    box_bottom = int(height * (1 - box_margin))
    
    box_color = "red"  
    box_thickness = 3
    draw.rectangle([(box_left, box_top), (box_right, box_bottom)], 
                  outline=box_color, width=box_thickness)
    
    try:
        font = ImageFont.truetype("arial.ttf", 10)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 30)
        except:
            font = ImageFont.load_default()

    class_text = f"Predicted: {prediction}"
    confidence_text = f"Confidence: {confidence:.1%}"
    
    text_margin_top = box_top - 70  
    
    text_bbox = draw.textbbox((box_left, text_margin_top), class_text, font=font)
    padding = 1  
    background_rect = (
        text_bbox[0] - padding,
        text_bbox[1] - padding,
        text_bbox[2] + padding,
        text_bbox[3] + padding
    )
    draw.rectangle(background_rect, fill="white")
    
    draw.text((box_left, text_margin_top), class_text, fill="black", font=font)
    
    conf_text_y = text_margin_top + 35  
    conf_bbox = draw.textbbox((box_left, conf_text_y), confidence_text, font=font)
    conf_background = (
        conf_bbox[0] - padding,
        conf_bbox[1] - padding,
        conf_bbox[2] + padding,
        conf_bbox[3] + padding
    )
    draw.rectangle(conf_background, fill="white")
    
    draw.text((box_left, conf_text_y), confidence_text, fill="black", font=font)
    
    return image

def predict_and_save_image(image_path, model, idx_to_class, device, output_dir="predicted_images"):
    """Predict class for a single image, draw bounding box, and save result"""
    os.makedirs(output_dir, exist_ok=True)
    
    original_image = Image.open(image_path).convert('RGB')
    
    image_tensor = transform(original_image)
    image_tensor = image_tensor.unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.softmax(output, dim=1)
        predicted_idx = output.argmax(dim=1).item()
        confidence = probabilities[0][predicted_idx].item()
        
    predicted_class = idx_to_class[predicted_idx]
    
    annotated_image = draw_prediction_box(original_image, predicted_class, confidence)
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"predicted_{timestamp}.jpg"
    output_path = os.path.join(output_dir, output_filename)
    
    annotated_image.save(output_path, quality=95)
    
    return {
        'class': predicted_class,
        'confidence': confidence,
        'index': predicted_idx,
        'saved_path': output_path
    }

if __name__ == "__main__":
    data_dir = 'dataset-original/'
    model_path = 'checkpoint/Trash-Classification.ckpt' #Replace with your model
    
    model, idx_to_class, device = setup_classifier(data_dir, model_path)
    
    print("Available classes:", list(idx_to_class.values()))
    
    image_path = 'glass.jpg' #replace with your image path
    result = predict_and_save_image(image_path, model, idx_to_class, device)
    
    print(f"Predicted Class: {result['class']}")
    print(f"Confidence: {result['confidence']:.2%}")
    print(f"Annotated image saved to: {result['saved_path']}")

Available classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Predicted Class: glass
Confidence: 92.41%
Annotated image saved to: predicted_images/predicted_20250109_212854.jpg
